In [1]:
# Imports
import random
import string
import tempfile
import shutil
import subprocess as sp
import os

from PIL import Image
from torch import nn as nn
from torch import Generator
from torch.utils.data import random_split,DataLoader
# TODO: Currently assumes that pytest runs from project root
from util.car_dataset import check_dir, check_file, CarDataset, DatasetError
from networks.common import *
from networks.CNNbase import CNNBasic

In [2]:
# Helper Functions
def rand_string(length=5):
    return ''.join(random.choices(string.ascii_letters +
        string.digits, k=length))

def loaded_dataset(temp_dir):
    '''
    Loads the car_dataset and returns the CarDataset object.
    '''
    image_folder_list = ['0_Frames','1_Frames','2_Frames','3_Frames']
    label_file_list = ['0.txt','1.txt','2.txt','3.txt']
    #image_folder_list = ['0_Frames']
    #label_file_list = ['0.txt']
    car_data = CarDataset(temp_dir,image_folder_list,label_file_list)
    car_data.drop_nan()

    return car_data

def dataset_loader(loaded_dataset):
    length = len(loaded_dataset)
    split_set = random_split(loaded_dataset,[round(0.80*length)
                ,round(0.20*length)],generator=Generator().manual_seed(42))
    train_loader = DataLoader(split_set[0],batch_size=20)
    test_loader = DataLoader(split_set[1],batch_size=1)
    '''
    Lowering values to sane levels to help run tests on github
    runners. Ideally on local GPU (4GB) 80/20 split with batch size of
    30 works well.
    '''
    return (train_loader,test_loader)

def network():
    network = CNNBasic()
    network = network.to(get_device())
    network.double()
    return network

In [3]:
def loaded_test_dataset(temp_dir):
    '''
    Loads the car_dataset and returns the CarDataset object.
    '''
    #image_folder_list = ['0_Frames','1_Frames','2_Frames','3_Frames']
    #label_file_list = ['0.txt','1.txt','2.txt','3.txt']
    image_folder_list = ['4_Frames']
    label_file_list = ['4.txt']
    car_data = CarDataset(temp_dir,image_folder_list,label_file_list)
    car_data.drop_nan()

    return car_data

In [4]:
root = os.getcwd()
print(os.getcwd())

/home/akshay.viswakumar/dl-based-sensor-calib


In [5]:
# Initialize Network
net = network()

In [6]:
# Load Checkpoint
epoch,loss =  load_model(net,net.optimizer,os.path.join(root,"checkpoint-0.pt"))

In [7]:
print("Trained Till Epoch : {}".format(epoch))
print("Training Loss : {}".format(loss))

Trained Till Epoch : 0
Training Loss : 0.0015060330042615533


In [8]:
new_test = loaded_test_dataset(os.path.join(root,"dummy"))

In [9]:
new_test_loader = DataLoader(new_test,batch_size=1)

In [10]:
run_inference(net,new_test_loader,get_device())

Num = 1109
Len = 1109
Test Set Average Loss 0.1197


In [12]:
# Single inference
samp = new_test[0]
out = run_inference_single(net,samp,get_device())

Num = 1
Len = 1
Test Set Average Loss 0.0001
Real = tensor([[0.0128, 0.0555]], device='cuda:0', dtype=torch.float64)
Pred = tensor([[0.0224, 0.0653]], device='cuda:0', dtype=torch.float64)
